In [5]:
import sys
sys.path.append("../Lib")
import utils as ut
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [6]:
# Obteniendo el conjunto de datos
df_reviews = pd.read_csv('../Data/csv/reviews_clean.csv', encoding='utf-8')
df_games = pd.read_csv('../Datasets/Clean/steam_games.csv.gz', encoding='utf-8')
df_items = pd.read_csv('../Datasets/Clean/users_items.csv.gz', encoding='utf-8')

### Model

Ahora, seleccionaremos variables para nuestros modelos de ML. 

Modelo 1: El modelo debe tener una relación ítem-ítem, es decir, se toma un ítem, en base a lo similar que sea ese ítem al resto se recomiendan similares. Nuestros artículos son juegos, por lo que los relacionaremos por género. 

Modelo 2: El sistema de recomendación debe aplicar el filtro usuario-elemento, es decir, tomar un usuario, buscar usuarios similares y recomendar artículos que les hayan gustado a esos usuarios similares. 

Luego tomaremos la relación entre los usuarios, sus recomendaciones y los juegos jugados por usuarios con características similares. Finalmente, vamos a unir las tablas de modelos en una sola tabla de modelos para cargar en nuestro archivo de modelos de ceromendaciones.

In [7]:
df_games

,publisher,genres,item_name,price,early_access,item_id,developer,release_year
0,Kotoshiro,Action,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
1,Kotoshiro,Casual,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
2,Kotoshiro,Indie,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
3,Kotoshiro,Simulation,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
4,Kotoshiro,Strategy,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
...,...,...,...,...,...,...,...,...
84206,NaN,Adventure,Maze Run VR,4.99,True,681550,NaN,no data
84207,NaN,Indie,Maze Run VR,4.99,True,681550,NaN,no data
84208,NaN,Action,Maze Run VR,4.99,True,681550,NaN,no data
84209,NaN,Simulation,Maze Run VR,4.99,True,681550,NaN,no data


### Model 1

In [8]:
df_model1 = df_games[['item_id', 'item_name', 'genres']]
df_model1

,item_id,item_name,genres
0,761140,Lost Summoner Kitty,Action
1,761140,Lost Summoner Kitty,Casual
2,761140,Lost Summoner Kitty,Indie
3,761140,Lost Summoner Kitty,Simulation
4,761140,Lost Summoner Kitty,Strategy
...,...,...,...
84206,681550,Maze Run VR,Adventure
84207,681550,Maze Run VR,Indie
84208,681550,Maze Run VR,Action
84209,681550,Maze Run VR,Simulation


In [9]:
df_model1.drop_duplicates(inplace=True)
df_model1

,item_id,item_name,genres
0,761140,Lost Summoner Kitty,Action
1,761140,Lost Summoner Kitty,Casual
2,761140,Lost Summoner Kitty,Indie
3,761140,Lost Summoner Kitty,Simulation
4,761140,Lost Summoner Kitty,Strategy
...,...,...,...
84206,681550,Maze Run VR,Adventure
84207,681550,Maze Run VR,Indie
84208,681550,Maze Run VR,Action
84209,681550,Maze Run VR,Simulation


In [10]:
ut.data_summ(df_model1)


Total rows:  84200

Total full null rows:  0


,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,item_id,[<class 'int'>],84200,0.00,0
1,item_name,"[<class 'str'>, <class 'float'>]",84198,0.00,2
2,genres,"[<class 'str'>, <class 'float'>]",84063,0.16,137


In [11]:
def summary(df_model1):
    print(f'data shape: {df_model1.shape}')
    summ = pd.DataFrame(df_model1.dtypes, columns=['data type'])
    summ['#missing'] = df_model1.isnull().sum()
    summ['%missing'] = df_model1.isnull().sum()*100/len(df_model1)
    summ['#unique'] = df_model1.nunique()
    desc = pd.DataFrame(df_model1.describe(include='all').T)
    summ['min'] = desc['min']
    summ['max'] = desc['max']
    return summ

summary(df_model1)

data shape: (84200, 3)


,data type,#missing,%missing,#unique,min,max
item_id,int64,0,0.000000,32019,10.0,2028850.0
item_name,object,2,0.002375,32018,NaN,NaN
genres,object,137,0.162708,256,NaN,NaN


### Model 2

In [12]:
df_auxr = df_reviews[['user_id', 'recommend', 'sentiment_analysis']]
df_auxi = df_items[['item_id', 'item_name', 'user_id']]
df_model2 = df_auxi.merge(df_auxr, on='user_id', how='inner')
df_model2

,item_id,item_name,user_id,recommend,sentiment_analysis
0,10,Counter-Strike,76561197970982479,True,2
1,10,Counter-Strike,76561197970982479,True,2
2,10,Counter-Strike,76561197970982479,True,2
3,30,Day of Defeat,76561197970982479,True,2
4,30,Day of Defeat,76561197970982479,True,2
...,...,...,...,...,...
4793069,400040,ShareX,76561198312638244,True,2
4793070,379720,DOOM,76561198312638244,True,2
4793071,379720,DOOM,76561198312638244,True,2
4793072,379720,DOOM,76561198312638244,True,2


In [13]:
df_model2.drop_duplicates(inplace=True)
df_model2

,item_id,item_name,user_id,recommend,sentiment_analysis
0,10,Counter-Strike,76561197970982479,True,2
3,30,Day of Defeat,76561197970982479,True,2
6,300,Day of Defeat: Source,76561197970982479,True,2
9,240,Counter-Strike: Source,76561197970982479,True,2
12,3830,Psychonauts,76561197970982479,True,2
...,...,...,...,...,...
4793054,238010,Deus Ex: Human Revolution - Director's Cut,76561198312638244,True,2
4793058,730,Counter-Strike: Global Offensive,76561198312638244,True,2
4793062,362890,Black Mesa,76561198312638244,True,2
4793066,400040,ShareX,76561198312638244,True,2


In [14]:
row = df_model2[df_model2['item_name'] == 'Counter-Strike']
row

,item_id,item_name,user_id,recommend,sentiment_analysis
0,10,Counter-Strike,76561197970982479,True,2
2805,10,Counter-Strike,doctr,True,2
2809,10,Counter-Strike,doctr,True,0
6785,10,Counter-Strike,WeiEDKrSat,True,1
7284,10,Counter-Strike,death-hunter,True,2
...,...,...,...,...,...
4790387,10,Counter-Strike,K1NGCJS,True,1
4790389,10,Counter-Strike,K1NGCJS,False,0
4791118,10,Counter-Strike,76561198208507532,True,0
4791119,10,Counter-Strike,76561198208507532,True,2


In [15]:
df_model2 = df_model2.drop_duplicates(subset=['item_id', 'user_id'], keep= 'first')
df_model2

,item_id,item_name,user_id,recommend,sentiment_analysis
0,10,Counter-Strike,76561197970982479,True,2
3,30,Day of Defeat,76561197970982479,True,2
6,300,Day of Defeat: Source,76561197970982479,True,2
9,240,Counter-Strike: Source,76561197970982479,True,2
12,3830,Psychonauts,76561197970982479,True,2
...,...,...,...,...,...
4793054,238010,Deus Ex: Human Revolution - Director's Cut,76561198312638244,True,2
4793058,730,Counter-Strike: Global Offensive,76561198312638244,True,2
4793062,362890,Black Mesa,76561198312638244,True,2
4793066,400040,ShareX,76561198312638244,True,2


Vamos a crear una nueva columna llamada 'Rating' donde clasificaremos los juegos según la recomendación dada por los usuarios y el análisis de sentimiento dado al juego respectivo.

In [16]:
def rating_def(row):
    '''
    Calculates a rating based on sentiment analysis and game review recommendations made by users.

    Parameters:
    row (dict): A dictionary containing the following keys:
        - "sentiment_analysis" (int): The sentiment analysis score (0, 1 or 2).
        - "recommend" (bool): Indicates whether the reviews recommend.

    Return:
        int or None: The rating calculated as an integer between 1 and 5, or None if the inputs are invalid.
    '''
    if row["sentiment_analysis"] == 0 and not row["recommend"]:
        return 1
    elif row["sentiment_analysis"] == 0 and row["recommend"]:
        return 1
    elif row["sentiment_analysis"] == 1 and not row["recommend"]:
        return 2
    elif row["sentiment_analysis"] == 1 and row["recommend"]:
        return 3
    elif row["sentiment_analysis"] == 2 and not row["recommend"]:
        return 4
    elif row["sentiment_analysis"] == 2 and row["recommend"]:
        return 5
    else:
        return None

In [17]:
df_model2['rating'] = df_model2.apply(rating_def, axis=1)
df_model2.drop(columns=['recommend', 'sentiment_analysis'], inplace=True)
df_model2

,item_id,item_name,user_id,rating
0,10,Counter-Strike,76561197970982479,5
3,30,Day of Defeat,76561197970982479,5
6,300,Day of Defeat: Source,76561197970982479,5
9,240,Counter-Strike: Source,76561197970982479,5
12,3830,Psychonauts,76561197970982479,5
...,...,...,...,...
4793054,238010,Deus Ex: Human Revolution - Director's Cut,76561198312638244,5
4793058,730,Counter-Strike: Global Offensive,76561198312638244,5
4793062,362890,Black Mesa,76561198312638244,5
4793066,400040,ShareX,76561198312638244,5


Uniendo Model 1 y Model 2

In [18]:
model = df_model1.merge(df_model2, on='item_name', how='inner')
model

,item_id_x,item_name,genres,item_id_y,user_id,rating
0,282010,Carmageddon Max Pack,Action,282010,UTNerd24,1
1,282010,Carmageddon Max Pack,Action,282010,saint556,5
2,282010,Carmageddon Max Pack,Action,282010,chidvd,5
3,282010,Carmageddon Max Pack,Action,282010,aerpub,1
4,282010,Carmageddon Max Pack,Action,282010,76561198001688490,5
...,...,...,...,...,...,...
3382627,80,Counter-Strike: Condition Zero,Action,80,jazzecks,5
3382628,80,Counter-Strike: Condition Zero,Action,80,lachwe,5
3382629,80,Counter-Strike: Condition Zero,Action,80,danebuchanan,5
3382630,80,Counter-Strike: Condition Zero,Action,80,76561198208507532,1


In [19]:
model.drop('item_id_y', axis=1, inplace=True)
model.rename(columns={'item_id_x': 'item_id'}, inplace=True)
model.drop_duplicates(inplace=True)
model

,item_id,item_name,genres,user_id,rating
0,282010,Carmageddon Max Pack,Action,UTNerd24,1
1,282010,Carmageddon Max Pack,Action,saint556,5
2,282010,Carmageddon Max Pack,Action,chidvd,5
3,282010,Carmageddon Max Pack,Action,aerpub,1
4,282010,Carmageddon Max Pack,Action,76561198001688490,5
...,...,...,...,...,...
3382627,80,Counter-Strike: Condition Zero,Action,jazzecks,5
3382628,80,Counter-Strike: Condition Zero,Action,lachwe,5
3382629,80,Counter-Strike: Condition Zero,Action,danebuchanan,5
3382630,80,Counter-Strike: Condition Zero,Action,76561198208507532,1


In [20]:
ut.data_summ(model)


Total rows:  3382339

Total full null rows:  0


,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,item_id,[<class 'int'>],3382339,0.0,0
1,item_name,[<class 'str'>],3382339,0.0,0
2,genres,[<class 'str'>],3382339,0.0,0
3,user_id,[<class 'str'>],3382339,0.0,0
4,rating,[<class 'int'>],3382339,0.0,0


In [21]:
dfs = [model]
# Nombres correspondientes a cada DataFrame
names = ['model']

ut.save_to_csv(dfs, names)

DataFrame 'model' saved as '../Data/csv/model.csv'


### Model

Ahora, seleccionaremos variables para nuestros modelos de ML. 

Modelo 1: El modelo debe tener una relación ítem-ítem, es decir, se toma un ítem, en base a lo similar que sea ese ítem al resto se recomiendan similares. Nuestros artículos son juegos, por lo que los relacionaremos por género. 

Modelo 2: El sistema de recomendación debe aplicar el filtro usuario-elemento, es decir, tomar un usuario, buscar usuarios similares y recomendar artículos que les hayan gustado a esos usuarios similares. 

Luego tomaremos la relación entre los usuarios, sus recomendaciones y los juegos jugados por usuarios con características similares. Finalmente, vamos a unir las tablas de modelos en una sola tabla de modelos para cargar en nuestro archivo de modelos de ceromendaciones.

In [22]:
df_games

,publisher,genres,item_name,price,early_access,item_id,developer,release_year
0,Kotoshiro,Action,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
1,Kotoshiro,Casual,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
2,Kotoshiro,Indie,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
3,Kotoshiro,Simulation,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
4,Kotoshiro,Strategy,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
...,...,...,...,...,...,...,...,...
84206,NaN,Adventure,Maze Run VR,4.99,True,681550,NaN,no data
84207,NaN,Indie,Maze Run VR,4.99,True,681550,NaN,no data
84208,NaN,Action,Maze Run VR,4.99,True,681550,NaN,no data
84209,NaN,Simulation,Maze Run VR,4.99,True,681550,NaN,no data


### Model 1

In [23]:
df_model1 = df_games[['item_id', 'item_name', 'genres']]
df_model1

,item_id,item_name,genres
0,761140,Lost Summoner Kitty,Action
1,761140,Lost Summoner Kitty,Casual
2,761140,Lost Summoner Kitty,Indie
3,761140,Lost Summoner Kitty,Simulation
4,761140,Lost Summoner Kitty,Strategy
...,...,...,...
84206,681550,Maze Run VR,Adventure
84207,681550,Maze Run VR,Indie
84208,681550,Maze Run VR,Action
84209,681550,Maze Run VR,Simulation


In [24]:
df_model1.drop_duplicates(inplace=True)
df_model1

,item_id,item_name,genres
0,761140,Lost Summoner Kitty,Action
1,761140,Lost Summoner Kitty,Casual
2,761140,Lost Summoner Kitty,Indie
3,761140,Lost Summoner Kitty,Simulation
4,761140,Lost Summoner Kitty,Strategy
...,...,...,...
84206,681550,Maze Run VR,Adventure
84207,681550,Maze Run VR,Indie
84208,681550,Maze Run VR,Action
84209,681550,Maze Run VR,Simulation


In [25]:
ut.data_summ(df_model1)


Total rows:  84200

Total full null rows:  0


,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,item_id,[<class 'int'>],84200,0.00,0
1,item_name,"[<class 'str'>, <class 'float'>]",84198,0.00,2
2,genres,"[<class 'str'>, <class 'float'>]",84063,0.16,137


In [26]:
def summary(df_model1):
    print(f'data shape: {df_model1.shape}')
    summ = pd.DataFrame(df_model1.dtypes, columns=['data type'])
    summ['#missing'] = df_model1.isnull().sum()
    summ['%missing'] = df_model1.isnull().sum()*100/len(df_model1)
    summ['#unique'] = df_model1.nunique()
    desc = pd.DataFrame(df_model1.describe(include='all').T)
    summ['min'] = desc['min']
    summ['max'] = desc['max']
    return summ

summary(df_model1)

data shape: (84200, 3)


,data type,#missing,%missing,#unique,min,max
item_id,int64,0,0.000000,32019,10.0,2028850.0
item_name,object,2,0.002375,32018,NaN,NaN
genres,object,137,0.162708,256,NaN,NaN


### Model 2

In [27]:
df_auxr = df_reviews[['user_id', 'recommend', 'sentiment_analysis']]
df_auxi = df_items[['item_id', 'item_name', 'user_id']]
df_model2 = df_auxi.merge(df_auxr, on='user_id', how='inner')
df_model2

,item_id,item_name,user_id,recommend,sentiment_analysis
0,10,Counter-Strike,76561197970982479,True,2
1,10,Counter-Strike,76561197970982479,True,2
2,10,Counter-Strike,76561197970982479,True,2
3,30,Day of Defeat,76561197970982479,True,2
4,30,Day of Defeat,76561197970982479,True,2
...,...,...,...,...,...
4793069,400040,ShareX,76561198312638244,True,2
4793070,379720,DOOM,76561198312638244,True,2
4793071,379720,DOOM,76561198312638244,True,2
4793072,379720,DOOM,76561198312638244,True,2


In [28]:
df_model2.drop_duplicates(inplace=True)
df_model2

,item_id,item_name,user_id,recommend,sentiment_analysis
0,10,Counter-Strike,76561197970982479,True,2
3,30,Day of Defeat,76561197970982479,True,2
6,300,Day of Defeat: Source,76561197970982479,True,2
9,240,Counter-Strike: Source,76561197970982479,True,2
12,3830,Psychonauts,76561197970982479,True,2
...,...,...,...,...,...
4793054,238010,Deus Ex: Human Revolution - Director's Cut,76561198312638244,True,2
4793058,730,Counter-Strike: Global Offensive,76561198312638244,True,2
4793062,362890,Black Mesa,76561198312638244,True,2
4793066,400040,ShareX,76561198312638244,True,2


In [29]:
row = df_model2[df_model2['item_name'] == 'Counter-Strike']
row

,item_id,item_name,user_id,recommend,sentiment_analysis
0,10,Counter-Strike,76561197970982479,True,2
2805,10,Counter-Strike,doctr,True,2
2809,10,Counter-Strike,doctr,True,0
6785,10,Counter-Strike,WeiEDKrSat,True,1
7284,10,Counter-Strike,death-hunter,True,2
...,...,...,...,...,...
4790387,10,Counter-Strike,K1NGCJS,True,1
4790389,10,Counter-Strike,K1NGCJS,False,0
4791118,10,Counter-Strike,76561198208507532,True,0
4791119,10,Counter-Strike,76561198208507532,True,2


In [30]:
df_model2 = df_model2.drop_duplicates(subset=['item_id', 'user_id'], keep= 'first')
df_model2

,item_id,item_name,user_id,recommend,sentiment_analysis
0,10,Counter-Strike,76561197970982479,True,2
3,30,Day of Defeat,76561197970982479,True,2
6,300,Day of Defeat: Source,76561197970982479,True,2
9,240,Counter-Strike: Source,76561197970982479,True,2
12,3830,Psychonauts,76561197970982479,True,2
...,...,...,...,...,...
4793054,238010,Deus Ex: Human Revolution - Director's Cut,76561198312638244,True,2
4793058,730,Counter-Strike: Global Offensive,76561198312638244,True,2
4793062,362890,Black Mesa,76561198312638244,True,2
4793066,400040,ShareX,76561198312638244,True,2


Vamos a crear una nueva columna llamada 'Rating' donde clasificaremos los juegos según la recomendación dada por los usuarios y el análisis de sentimiento dado al juego respectivo.

In [31]:
def rating_def(row):
    '''
    Calculates a rating based on sentiment analysis and game review recommendations made by users.

    Parameters:
    row (dict): A dictionary containing the following keys:
        - "sentiment_analysis" (int): The sentiment analysis score (0, 1 or 2).
        - "recommend" (bool): Indicates whether the reviews recommend.

    Return:
        int or None: The rating calculated as an integer between 1 and 5, or None if the inputs are invalid.
    '''
    if row["sentiment_analysis"] == 0 and not row["recommend"]:
        return 1
    elif row["sentiment_analysis"] == 0 and row["recommend"]:
        return 1
    elif row["sentiment_analysis"] == 1 and not row["recommend"]:
        return 2
    elif row["sentiment_analysis"] == 1 and row["recommend"]:
        return 3
    elif row["sentiment_analysis"] == 2 and not row["recommend"]:
        return 4
    elif row["sentiment_analysis"] == 2 and row["recommend"]:
        return 5
    else:
        return None

In [32]:
df_model2['rating'] = df_model2.apply(rating_def, axis=1)
df_model2.drop(columns=['recommend', 'sentiment_analysis'], inplace=True)
df_model2

,item_id,item_name,user_id,rating
0,10,Counter-Strike,76561197970982479,5
3,30,Day of Defeat,76561197970982479,5
6,300,Day of Defeat: Source,76561197970982479,5
9,240,Counter-Strike: Source,76561197970982479,5
12,3830,Psychonauts,76561197970982479,5
...,...,...,...,...
4793054,238010,Deus Ex: Human Revolution - Director's Cut,76561198312638244,5
4793058,730,Counter-Strike: Global Offensive,76561198312638244,5
4793062,362890,Black Mesa,76561198312638244,5
4793066,400040,ShareX,76561198312638244,5


Uniendo Model 1 y Model 2

In [33]:
model = df_model1.merge(df_model2, on='item_name', how='inner')
model

,item_id_x,item_name,genres,item_id_y,user_id,rating
0,282010,Carmageddon Max Pack,Action,282010,UTNerd24,1
1,282010,Carmageddon Max Pack,Action,282010,saint556,5
2,282010,Carmageddon Max Pack,Action,282010,chidvd,5
3,282010,Carmageddon Max Pack,Action,282010,aerpub,1
4,282010,Carmageddon Max Pack,Action,282010,76561198001688490,5
...,...,...,...,...,...,...
3382627,80,Counter-Strike: Condition Zero,Action,80,jazzecks,5
3382628,80,Counter-Strike: Condition Zero,Action,80,lachwe,5
3382629,80,Counter-Strike: Condition Zero,Action,80,danebuchanan,5
3382630,80,Counter-Strike: Condition Zero,Action,80,76561198208507532,1


In [34]:
model.drop('item_id_y', axis=1, inplace=True)
model.rename(columns={'item_id_x': 'item_id'}, inplace=True)
model.drop_duplicates(inplace=True)
model

,item_id,item_name,genres,user_id,rating
0,282010,Carmageddon Max Pack,Action,UTNerd24,1
1,282010,Carmageddon Max Pack,Action,saint556,5
2,282010,Carmageddon Max Pack,Action,chidvd,5
3,282010,Carmageddon Max Pack,Action,aerpub,1
4,282010,Carmageddon Max Pack,Action,76561198001688490,5
...,...,...,...,...,...
3382627,80,Counter-Strike: Condition Zero,Action,jazzecks,5
3382628,80,Counter-Strike: Condition Zero,Action,lachwe,5
3382629,80,Counter-Strike: Condition Zero,Action,danebuchanan,5
3382630,80,Counter-Strike: Condition Zero,Action,76561198208507532,1


In [35]:
ut.data_summ(model)


Total rows:  3382339

Total full null rows:  0


,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,item_id,[<class 'int'>],3382339,0.0,0
1,item_name,[<class 'str'>],3382339,0.0,0
2,genres,[<class 'str'>],3382339,0.0,0
3,user_id,[<class 'str'>],3382339,0.0,0
4,rating,[<class 'int'>],3382339,0.0,0


In [36]:
dfs = [model]
# Nombres correspondientes a cada DataFrame
names = ['model']

ut.save_to_csv(dfs, names)

DataFrame 'model' saved as '../Data/csv/model.csv'


In [37]:
df_games

,publisher,genres,item_name,price,early_access,item_id,developer,release_year
0,Kotoshiro,Action,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
1,Kotoshiro,Casual,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
2,Kotoshiro,Indie,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
3,Kotoshiro,Simulation,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
4,Kotoshiro,Strategy,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
...,...,...,...,...,...,...,...,...
84206,NaN,Adventure,Maze Run VR,4.99,True,681550,NaN,no data
84207,NaN,Indie,Maze Run VR,4.99,True,681550,NaN,no data
84208,NaN,Action,Maze Run VR,4.99,True,681550,NaN,no data
84209,NaN,Simulation,Maze Run VR,4.99,True,681550,NaN,no data


### Model 1

In [38]:
df_model1 = df_games[['item_id', 'item_name', 'genres']]
df_model1

,item_id,item_name,genres
0,761140,Lost Summoner Kitty,Action
1,761140,Lost Summoner Kitty,Casual
2,761140,Lost Summoner Kitty,Indie
3,761140,Lost Summoner Kitty,Simulation
4,761140,Lost Summoner Kitty,Strategy
...,...,...,...
84206,681550,Maze Run VR,Adventure
84207,681550,Maze Run VR,Indie
84208,681550,Maze Run VR,Action
84209,681550,Maze Run VR,Simulation


In [39]:
df_model1.drop_duplicates(inplace=True)
df_model1

,item_id,item_name,genres
0,761140,Lost Summoner Kitty,Action
1,761140,Lost Summoner Kitty,Casual
2,761140,Lost Summoner Kitty,Indie
3,761140,Lost Summoner Kitty,Simulation
4,761140,Lost Summoner Kitty,Strategy
...,...,...,...
84206,681550,Maze Run VR,Adventure
84207,681550,Maze Run VR,Indie
84208,681550,Maze Run VR,Action
84209,681550,Maze Run VR,Simulation


In [40]:
ut.data_summ(df_model1)


Total rows:  84200

Total full null rows:  0


,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,item_id,[<class 'int'>],84200,0.00,0
1,item_name,"[<class 'str'>, <class 'float'>]",84198,0.00,2
2,genres,"[<class 'str'>, <class 'float'>]",84063,0.16,137


In [41]:
def summary(df_model1):
    print(f'data shape: {df_model1.shape}')
    summ = pd.DataFrame(df_model1.dtypes, columns=['data type'])
    summ['#missing'] = df_model1.isnull().sum()
    summ['%missing'] = df_model1.isnull().sum()*100/len(df_model1)
    summ['#unique'] = df_model1.nunique()
    desc = pd.DataFrame(df_model1.describe(include='all').T)
    summ['min'] = desc['min']
    summ['max'] = desc['max']
    return summ

summary(df_model1)

data shape: (84200, 3)


,data type,#missing,%missing,#unique,min,max
item_id,int64,0,0.000000,32019,10.0,2028850.0
item_name,object,2,0.002375,32018,NaN,NaN
genres,object,137,0.162708,256,NaN,NaN


### Model 2

In [42]:
df_auxr = df_reviews[['user_id', 'recommend', 'sentiment_analysis']]
df_auxi = df_items[['item_id', 'item_name', 'user_id']]
df_model2 = df_auxi.merge(df_auxr, on='user_id', how='inner')
df_model2

,item_id,item_name,user_id,recommend,sentiment_analysis
0,10,Counter-Strike,76561197970982479,True,2
1,10,Counter-Strike,76561197970982479,True,2
2,10,Counter-Strike,76561197970982479,True,2
3,30,Day of Defeat,76561197970982479,True,2
4,30,Day of Defeat,76561197970982479,True,2
...,...,...,...,...,...
4793069,400040,ShareX,76561198312638244,True,2
4793070,379720,DOOM,76561198312638244,True,2
4793071,379720,DOOM,76561198312638244,True,2
4793072,379720,DOOM,76561198312638244,True,2


In [43]:
df_model2.drop_duplicates(inplace=True)
df_model2

,item_id,item_name,user_id,recommend,sentiment_analysis
0,10,Counter-Strike,76561197970982479,True,2
3,30,Day of Defeat,76561197970982479,True,2
6,300,Day of Defeat: Source,76561197970982479,True,2
9,240,Counter-Strike: Source,76561197970982479,True,2
12,3830,Psychonauts,76561197970982479,True,2
...,...,...,...,...,...
4793054,238010,Deus Ex: Human Revolution - Director's Cut,76561198312638244,True,2
4793058,730,Counter-Strike: Global Offensive,76561198312638244,True,2
4793062,362890,Black Mesa,76561198312638244,True,2
4793066,400040,ShareX,76561198312638244,True,2


In [44]:
row = df_model2[df_model2['item_name'] == 'Counter-Strike']
row

,item_id,item_name,user_id,recommend,sentiment_analysis
0,10,Counter-Strike,76561197970982479,True,2
2805,10,Counter-Strike,doctr,True,2
2809,10,Counter-Strike,doctr,True,0
6785,10,Counter-Strike,WeiEDKrSat,True,1
7284,10,Counter-Strike,death-hunter,True,2
...,...,...,...,...,...
4790387,10,Counter-Strike,K1NGCJS,True,1
4790389,10,Counter-Strike,K1NGCJS,False,0
4791118,10,Counter-Strike,76561198208507532,True,0
4791119,10,Counter-Strike,76561198208507532,True,2


In [45]:
df_model2 = df_model2.drop_duplicates(subset=['item_id', 'user_id'], keep= 'first')
df_model2

,item_id,item_name,user_id,recommend,sentiment_analysis
0,10,Counter-Strike,76561197970982479,True,2
3,30,Day of Defeat,76561197970982479,True,2
6,300,Day of Defeat: Source,76561197970982479,True,2
9,240,Counter-Strike: Source,76561197970982479,True,2
12,3830,Psychonauts,76561197970982479,True,2
...,...,...,...,...,...
4793054,238010,Deus Ex: Human Revolution - Director's Cut,76561198312638244,True,2
4793058,730,Counter-Strike: Global Offensive,76561198312638244,True,2
4793062,362890,Black Mesa,76561198312638244,True,2
4793066,400040,ShareX,76561198312638244,True,2


Vamos a crear una nueva columna llamada 'Rating' donde clasificaremos los juegos según la recomendación dada por los usuarios y el análisis de sentimiento dado al juego respectivo.

In [46]:
def rating_def(row):
    '''
    Calculates a rating based on sentiment analysis and game review recommendations made by users.

    Parameters:
    row (dict): A dictionary containing the following keys:
        - "sentiment_analysis" (int): The sentiment analysis score (0, 1 or 2).
        - "recommend" (bool): Indicates whether the reviews recommend.

    Return:
        int or None: The rating calculated as an integer between 1 and 5, or None if the inputs are invalid.
    '''
    if row["sentiment_analysis"] == 0 and not row["recommend"]:
        return 1
    elif row["sentiment_analysis"] == 0 and row["recommend"]:
        return 1
    elif row["sentiment_analysis"] == 1 and not row["recommend"]:
        return 2
    elif row["sentiment_analysis"] == 1 and row["recommend"]:
        return 3
    elif row["sentiment_analysis"] == 2 and not row["recommend"]:
        return 4
    elif row["sentiment_analysis"] == 2 and row["recommend"]:
        return 5
    else:
        return None

In [47]:
df_model2['rating'] = df_model2.apply(rating_def, axis=1)
df_model2.drop(columns=['recommend', 'sentiment_analysis'], inplace=True)
df_model2

,item_id,item_name,user_id,rating
0,10,Counter-Strike,76561197970982479,5
3,30,Day of Defeat,76561197970982479,5
6,300,Day of Defeat: Source,76561197970982479,5
9,240,Counter-Strike: Source,76561197970982479,5
12,3830,Psychonauts,76561197970982479,5
...,...,...,...,...
4793054,238010,Deus Ex: Human Revolution - Director's Cut,76561198312638244,5
4793058,730,Counter-Strike: Global Offensive,76561198312638244,5
4793062,362890,Black Mesa,76561198312638244,5
4793066,400040,ShareX,76561198312638244,5


Uniendo Model 1 y Model 2

In [48]:
model = df_model1.merge(df_model2, on='item_name', how='inner')
model

,item_id_x,item_name,genres,item_id_y,user_id,rating
0,282010,Carmageddon Max Pack,Action,282010,UTNerd24,1
1,282010,Carmageddon Max Pack,Action,282010,saint556,5
2,282010,Carmageddon Max Pack,Action,282010,chidvd,5
3,282010,Carmageddon Max Pack,Action,282010,aerpub,1
4,282010,Carmageddon Max Pack,Action,282010,76561198001688490,5
...,...,...,...,...,...,...
3382627,80,Counter-Strike: Condition Zero,Action,80,jazzecks,5
3382628,80,Counter-Strike: Condition Zero,Action,80,lachwe,5
3382629,80,Counter-Strike: Condition Zero,Action,80,danebuchanan,5
3382630,80,Counter-Strike: Condition Zero,Action,80,76561198208507532,1


In [49]:
model.drop('item_id_y', axis=1, inplace=True)
model.rename(columns={'item_id_x': 'item_id'}, inplace=True)
model.drop_duplicates(inplace=True)
model

,item_id,item_name,genres,user_id,rating
0,282010,Carmageddon Max Pack,Action,UTNerd24,1
1,282010,Carmageddon Max Pack,Action,saint556,5
2,282010,Carmageddon Max Pack,Action,chidvd,5
3,282010,Carmageddon Max Pack,Action,aerpub,1
4,282010,Carmageddon Max Pack,Action,76561198001688490,5
...,...,...,...,...,...
3382627,80,Counter-Strike: Condition Zero,Action,jazzecks,5
3382628,80,Counter-Strike: Condition Zero,Action,lachwe,5
3382629,80,Counter-Strike: Condition Zero,Action,danebuchanan,5
3382630,80,Counter-Strike: Condition Zero,Action,76561198208507532,1


In [50]:
ut.data_summ(model)


Total rows:  3382339

Total full null rows:  0


,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,item_id,[<class 'int'>],3382339,0.0,0
1,item_name,[<class 'str'>],3382339,0.0,0
2,genres,[<class 'str'>],3382339,0.0,0
3,user_id,[<class 'str'>],3382339,0.0,0
4,rating,[<class 'int'>],3382339,0.0,0


In [51]:
dfs = [model]
# Nombres correspondientes a cada DataFrame
names = ['model']

ut.save_to_csv(dfs, names)

DataFrame 'model' saved as '../Data/csv/model.csv'
